
0. Intersection (all docId that contain all query terms)  
1. Calculate tf-idf for each query term with in each doc from step 0  
2. For each doc add up tf-idf (all query terms in doc)  
3. Rank highest to lowest, return doc id  

In [1]:
from HelperClass import InvertedIndex
from nltk.stem import PorterStemmer
from collections import defaultdict
import math
import re
import shelve

def calculate_idf(N, df):
    idf = math.log((N + 0.1) / (df + 0.1))
    return idf

def getTfIdf(tf, df, N):
    return tf * calculate_idf(N, df)

def stemQuery(query:str) -> list:
    stemmer = PorterStemmer()
    queryList = list()
    line = query.strip()
    if line != '':
        for aToken in re.split('[^a-z0-9]', line.lower()):
            if (aToken != ''):
                token = stemmer.stem(aToken)
                queryList.append(token)
    return queryList

def getDfDict(query:set, invIndex):
    queryDict = defaultdict(lambda: '')
    for token in query:
        if token in invIndex:
            queryDict[token] = len(invIndex['pos'][token]) #df
        else:
            queryDict[token] = 0
    return queryDict

def getDocIds(qDict:dict, invIndex) -> set:
    unionSet = set()
    for word in qDict:
        if len(unionSet) == 0:
            unionSet = set(invIndex['pos'][word].keys())
        else:
            unionSet = unionSet.intersection(set(invIndex['pos'][word].keys()))
    return unionSet

def getTfIdfDict(qDict:dict, invIndex, N) -> dict:
    tfIdfDict = defaultdict(lambda: 0)
    for docId in getDocIds(qDict, invIndex):
        for word in qDict:
            tf = len(invIndex['pos'][word][docId])
            tfIdfDict[docId] += getTfIdf(tf, qDict[word], N)
    
    #Sort based on tfIdf
    return {k: v for k, v in sorted(tfIdfDict.items(), key=lambda item: item[1], reverse=True)}

def getTopResults(idDict, limit):
    count = 0
    for docId in idDict:
        with shelve.open(f'DevShelve/Url', 'c') as shelf:
            print(shelf[str(docId)])
            count += 1
        if count >= limit: return

N = 55383
query = 'cristina lopes'
stemList = stemQuery(query)
invIndex = InvertedIndex()
invIndex.load(stemList,count=10, filePath='DevShelve') #10 partial indexes in shelve file
qDict = getDfDict(stemList,invIndex)
tfIdfDict = getTfIdfDict(qDict, invIndex, N)
# getTopResults(tfIdfDict,5)

In [2]:
import shelve
def getDocNum():
    with shelve.open(f'DevShelve/Index', 'c') as shelf:
        print(list(shelf.keys()))

getDocNum()

['index1', 'weight1', 'index2', 'weight2', 'index3', 'weight3', 'index4', 'weight4', 'index5', 'weight5', 'index6', 'weight6', 'index7', 'weight7', 'index8', 'weight8', 'index9', 'weight9', 'index10', 'weight10']


['index1', 'index2', 'index3', 'index4', 'index5', 'index6', 'index7', 'index8', 'index9', 'index10']